<a href="https://colab.research.google.com/gist/sadov/47fce086ec4bc5d6d7ec43668875c872/dateno_llm_context_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Пример использования Dateno LLM для решения прикладной задачи

Для проведения исследований в области данных с использованием системы Dateno предлагается интерфейс Jupiter Notebook (в данном случае: Google Collab). Представляется, что интерактивный документ представляемый такой платформой -- это наиболее близкая абстракция для таких специалистов, конечным продуктом работы которых являются report'ы, статьи и аналитические записки.

Как мы увидим на этом примере, процесс создания экспериментального модуля ориентированного на выборку и обработку данных из Dateno может быть разделён между предметным специалистом для подготовки контекстов запросов в ходе обычного взаимодействия с LLM-агентом в форме запрос/ответ и программистом, который готовит функции специфичные для данной задачи. После наработки опредлённой проблемно-специфической практики какая-то часть этой работы также может быть передана AI-агентам.

Хотим посмотреть как соотносится количество датасетов имеющих отношение к подоходному налогу в тех или иных странах на официальных национальных языках этих стран. Насколько широко используются в метаданных по этим датасетам заимствования из английского.

Для этого будем использовать LLM-агента `datenoio/dateno-search` с передачей ему соответствующего контекста запроса.

## Подготовка

Первым делом прописываем токен для доступа к [Hugging Face Spaces Dateno](https://huggingface.co/datenoio/spaces):

In [ ]:
HF_TOKEN='YOUR DATENO HF SPACE TOKEN'

Устанавливаем необходимые пакеты:

In [ ]:
!pip install git+https://github.com/datenoio/datenollm

  Cloning https://github.com/sadov/datenollm to /tmp/pip-req-build-ybevin1_
  Running command git clone --filter=blob:none --quiet https://github.com/sadov/datenollm /tmp/pip-req-build-ybevin1_
  Resolved https://github.com/sadov/datenollm to commit a6436592864fc4d9b34d42b4a78d510a5e3be968
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.3 MB/s eta 0:00:00
  Created wheel for datenollm: filename=datenollm-0.0.1-py3-none-any.whl size=26315 sha256=c1d44b072be7b8f49b87adfa7b810e0b3c075f471e87bfe833db58625dfa7186
  Stored in directory

Импортируем то что понадобится:

In [ ]:
import json
import pandas as pd

from datenollm.client import DatenoClient
from datenollm.jupiter_nb import (
    get_full_path, ask_llm,
    ChatWidget, DatenoSearchChatWidget,
    results_table, history2context,
    copy_test_data,
)

Проверим есть ли у нас доступ Google Disk'у. Если его нет, то последует запрос на выдачу разрешений для доступа к Google Account'у.

Google запросит очень много разрешений. Однако, если выставить только разрешение на доступ к файлам и каталогам, то подключения к Google Disk не происходит.

По умолчанию для сохранения в файлах истории и контекста мы используем только каталог /content/drive/MyDrive/colab_data/dateno/, ну или каталог указанный в переменной среды окружения `DRIVE_PATH`, ничего больше. См. https://github.com/datenoio/datenollm/blob/main/datenollm/file_utils.py

Если условие предоставления доступа к данным на персональном Google Disk'е является критическим, можно порекомендовать запускать данный Google Collab notebook на другом аккаунте Google, где важных данных нет.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


После предоставления доступа проверяем есть ли каталог /content/drive/MyDrive/colab_data/dateno/, если нет -- он будет создан. Для чистоты эксперимента можно удалить файлы которые имеются в данном каталоге или перенести их в другой каталог.

In [ ]:
get_full_path('')

'/content/drive/MyDrive/colab_data/dateno/'

Стартуем клиент для доступа к HF Space, где работает нужный нам LLM-агент. **М.б. придется позапускать несколько раз если будут ошибки** -- HF Space засыпает если долго не использовался и запускается опять при обращении к нему. При таком раскладе -- заводиться может пинка с третьего.

In [ ]:
client = DatenoClient('datenoio/dateno-search', hf_token=HF_TOKEN)

Loaded as API: https://datenoio-dateno-search.hf.space ✔


## Подготавливаем экспериментальный стенд

In [ ]:
# @title Определяем функции.

def ask_llm_for_country(client, country, context_file=None, history_file=None):
  if not country:
    return None, None, None, "Provide a country name"
  country = country.title()
  query=f'income taxes in {country} in all official national languages'
  print(f"Query: {query}")
  query_text, result_json, history, error = ask_llm(client, query,
                                                    context_file=context_file,
                                                    history_file=history_file)
  return query, result_json, history, error

def dateno_index_search(llm_response):
  if not llm_response or "queries" not in llm_response:
    return None, None, None, "Invalid LLM response format"

  if type(llm_response) is str:
    llm_response = json.loads(llm_response)

  # The primary query for the table title can be the query from the first filtered result
  if llm_response.get('queries') and len(llm_response["queries"]):
    primary_query_text = llm_response["queries"][0].get('query', '')
  else:
    return None, None, None, "Empty LLM response"

  filtered_queries = []
  for query_obj in llm_response["queries"]:
    # Check if filters exist and contain the required language filter
    if query_obj.get('filters'):
      lang_filter_exists = any(f for f in query_obj['filters'] if f and f.get('name') == 'source.langs.name')
      if lang_filter_exists:
        filtered_queries.append(query_obj)
        print(f"Subquery: {query_obj.get('query', '')} | Filters: {query_obj.get('filters', [])}")
      else:
          print(f"Skipping query due to missing language filter: {query_obj.get('query', '')}")
    else:
        print(f"Skipping query due to missing filters: {query_obj.get('query', '')}")

  if not filtered_queries:
      return None, [], [], "No queries with language filter found"

  # Create a new structure with only the filtered queries
  filtered_llm_response = {"queries": filtered_queries}

  # Convert the dictionary to a JSON string
  filtered_llm_response_json_string = json.dumps(filtered_llm_response)

  results = client.client.predict(llm_response=filtered_llm_response_json_string, api_name="/dateno_search")

  data = []
  # The results from predict should now correspond to the filtered_queries
  # We need to iterate through the results and match them with the original filtered queries
  # to extract the language and dataset count.
  # Assuming the order of results matches the order of filtered_queries
  for i, result in enumerate(results):
      query_info = filtered_queries[i] # Use the original filtered query info
      filters = query_info.get('filters', [])
      # We already know there is a language filter, so we can proceed
      lang_filter = next((f for f in filters if f and f.get('name') == 'source.langs.name'), None)
      if lang_filter:
          data.append({
              'Language': lang_filter['value'],
              'Number of datasets ': len(result['results']['hits']['hits'])
          })

  return primary_query_text, results, data, None

Проверяем как это работает.

In [ ]:
# @title Вводим интересующую нас страну:
country = "ceylon" # @param {"type":"string","placeholder":"Provide a country name"}

query_text, llm_response, history, error = ask_llm_for_country(client, country)
if error:
    raise ValueError(f"Error: {error}")

query, results, data, error = dateno_index_search(llm_response)
table = results_table(query, data)
table

Query: income taxes in Ceylon in all official national languages
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'
Skipping query due to missing filters: income taxes in Ceylon in all official national languages
Subquery: ආදායම් බදු | Filters: [{'name': 'source.countries.name', 'value': 'Sri Lanka'}, {'name': 'source.langs.name', 'value': 'Sinhala'}]
Subquery: வருமான வரிகள் | Filters: [{'name': 'source.countries.name', 'value': 'Sri Lanka'}, {'name': 'source.langs.name', 'value': 'Tamil'}]
Subquery: income taxes | Filters: [{'name': 'source.countries.name', 'value': 'Sri Lanka'}, {'name': 'source.langs.name', 'value': 'English'}]

Datasets for query 'income taxes in Ceylon in all official national languages':



,Language,Number of datasets
0,Sinhala,0
1,Tamil,0
2,English,500


Копируем подготовленные ранее контексты.

In [ ]:
copy_test_data()

## Эксперимент

А теперь смотрим сколько датасетов у нас есть на каких языках и насколько они используют заимствования из английского.

Проверяем этот запрос с переводом на выбранный язык и без перевода. Для этого мы имеем два разных контекста.

In [ ]:
# @title Вводим интересующую нас страну:
country = "switzerland" # @param {"type":"string","placeholder":"Provide a country name"}
if not country:
  raise ValueError("Provide a country name")
country = country.title()
query=f'income taxes in {country} in all official national languages'
print(f"Query: {query}")

Query: income taxes in Switzerland in all official national languages


Проверяем этот запрос с переводом на выбранный язык и без перевода.

In [ ]:
# @title Количество датасетов на нац. языках без перевода запроса на эти языки:
history_notr = "history-trans.json"
context_notr = "context-trans.json"

query_text, llm_response, history, error = ask_llm_for_country(client, country,
                                                               context_file=context_notr,
                                                               history_file=history_notr)
if error:
    raise ValueError(f"Error: {error}")

query, results, data, error = dateno_index_search(llm_response)
table = results_table(query, data)
table

Query: income taxes in Switzerland in all official national languages
context_file='/content/drive/MyDrive/colab_data/dateno/context-trans,json'
history_file='/content/drive/MyDrive/colab_data/dateno/history-trans.json'
Skipping query due to missing filters: income taxes in Switzerland
Subquery: income taxes | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'German'}]
Subquery: income taxes | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'French'}]
Subquery: income taxes | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'Italian'}]
Subquery: income taxes | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'Romansh'}]
Subquery: income taxes | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'E

,Language,Number of datasets
0,German,60
1,French,2
2,Italian,0
3,Romansh,0
4,English,407


Теперь смотрим какая ситуация у нас будет при выборке в которой основной запрос переведён на национальные языки. **Внимание: если очевидно, что агент не перевел запрос, а просто оставил его в исходном виде (как в предыдущей ячейке), просто выполните запрос в этой ячейке ещё раз -- скорее всего LLM исправится и осуществит перевод.**

In [ ]:
# @title Количество датасетов на нац. языках с переводом запроса на эти языки:
history_tr = "history+trans.json"
context_tr = "context+trans.json"

query_text, llm_response, history, error = ask_llm_for_country(client, country,
                                                               context_file=context_tr,
                                                               history_file=history_tr)
if error:
    raise ValueError(f"Error: {error}")

query, results, data, error = dateno_index_search(llm_response)
table = results_table(query, data)
table

Query: income taxes in Switzerland in all official national languages
context_file='/content/drive/MyDrive/colab_data/dateno/context+trans,json'
history_file='/content/drive/MyDrive/colab_data/dateno/history+trans.json'
Skipping query due to missing filters: income taxes in Switzerland in all official national languages
Subquery: Einkommensteuer | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'German'}]
Subquery: impôts sur le revenu | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'French'}]
Subquery: imposta sul reddito | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'Italian'}]
Subquery: tassa sin la renda | Filters: [{'name': 'source.countries.name', 'value': 'Switzerland'}, {'name': 'source.langs.name', 'value': 'Romansh'}]
Subquery: income taxes | Filters: [{'name': 'source.countries.name', 'value'

,Language,Number of datasets
0,German,0
1,French,500
2,Italian,0
3,Romansh,0
4,English,407


То есть можно сделать вывод: во франкоязычных кантонах Швейцарии стараются всё переводить на французский, а в германоязычных — довольно легко заимствуют англоязычную лексику, благо это та же германская группа языков 😉

## Как мы этого добились

Мы используем два разных контекста для выполнения перевода запроса на местные языки и для запросов без перевода.

Для того что бы создать файл нужного нам контекста мы посылаем запросы нашему LLM-агенту и проставляем пометки Like/Dislike если хотим какие-то из моментов нашего conversation'а пометить для дальнейшего использования в качестве контекста запроса. Продолжаем эту процедуру пока не получим желаемого результата.

**Внимание: если ответ не был получен или он не по каким-то соображениям не устраивает, попробуйте выполните запрос в этой ячейке ещё раз, возможно переформулировав и/или уточнив его.**

In [ ]:
# @title Вводим запрос:
query = "use all official langs of this country with translation of query to this langs" # @param {"type":"string","placeholder":"Provide a country name"}
history_file = "history.json" # @param {"type":"string","placeholder":"Enter History's file name"}

query_text, result_json, history, error = ask_llm(client, query, history_file=history_file)
if error:
    raise ValueError(f"Error: {error}")
else:
    rating_widget = DatenoSearchChatWidget(client, history_file=history_file)
    rwd = rating_widget.display()
    display(rwd)

context_file='/content/drive/MyDrive/colab_data/dateno/context,json'
history_file='/content/drive/MyDrive/colab_data/dateno/history.json'


Так можем посмотреть всю историю.

In [ ]:
rating_widget.display_history()

HTML(value='===============================================<br><strong>1. Question:</strong> <strong><em>incom…

Конвертируем историю в контекст:

In [ ]:
context_file='context.json'

history2context(history_file, context_file)

Вот что у нас получается:

In [ ]:
with open(get_full_path(context_file), 'r') as f:
  context_data = json.load(f)

rating_widget.display_history(context_data)


HTML(value='===============================================<br><strong>1. Question:</strong> <strong><em>incom…

Как видим, туда попали только помеченные нами фрагменты conersation'а с LLM-агентом. Теперь можем использовать этот файл контекста для выполнения тех или иных запросов. В нашем случае, напр. -- мы имеем два разных контекста для выполнения перевода запроса на местные языки и для запросов без перевода.

Далее эти файлы контекста можем передавать как аргументы функциям обращения к LLM-агентам.